In [1]:
import json
import os
import copy
from typing import Union, List, Tuple, Dict, DefaultDict, Callable, Any, Optional

import autogen
from autogen import config_list_from_json
from autogen.code_utils import extract_code
from autogen.math_utils import get_answer
from autogen.agentchat import Agent
from autogen.agentchat.contrib.agent_optimizer import AgentOptimizer
from autogen.agentchat.contrib.math_user_proxy_agent import MathUserProxyAgent

## MathUserProxy with function_call

This agent is a customized `MathUserProxy` inherits from its [parent class](https://github.com/microsoft/autogen/blob/main/autogen/agentchat/contrib/math_user_proxy_agent.py).

It supports using both `function_call` and python to solve math problems.

In [11]:
def is_termination_msg_mathchat(message):
    if isinstance(message, dict):
        message = message.get('content')
        if message is None: return False
    
    code_list = extract_code(message)
    contain_code = False
    for code in code_list:
        if code[0] == 'python':
            contain_code = True
            break
    
    if message.rstrip().find('TERMINATE') > 0:
        return True

    answer = get_answer(message)
    return contain_code and (answer is not None) and (answer != "")